In [1]:
from fforma import *
import pandas as pd
from fforma.train_models import trainBasicModels
import random
import matplotlib.pyplot as plt

In [2]:
def get_data(data = 'M4', freq = 'Monthly', frcy=12, val_periods=18, basic_models = [Naive(), SeasonalNaive(), RandomWalkDrift()],
               sample_sizes = [.1, .2, .3, .4, .5]):
    # Reding complete data
    if data == 'M4':
        data = pd.read_csv(f'data/data_m4/{freq}-train.csv')
        data_test = pd.read_csv(f'data/data_m4/{freq}-test.csv')

        # Train and test set
        ts_list_train = [ts[1:].dropna().values for idx, ts in data.iterrows()]
        ts_list_test = [ts[1:].dropna().values for idx, ts in data_test.iterrows()]
    else:
        freq_dict = {
            'Monthly': 'M'
        }
        
        freq_m3 = freq_dict[freq]
        data = pd.read_csv(f'data/data_m3/dataM3.csv').query('freq==@freq_m3')
        data_test = data.query('type=="test"')
        data = data.query('type=="train"')

        # Train and test set
        ts_list_train = [df['value'].values for idx, df in data.groupby('id')]
        ts_list_test = [df['value'].values for idx, df in data_test.groupby('id')]
        
    #Test periods
    test_periods = len(ts_list_test[0])
    
    # Validation set
    ts_train_for_val = [ts[:-val_periods] for ts in ts_list_train] 
    ts_val_list = [ts[-val_periods:] for ts in ts_list_train]
    
    # Prediction for validation set and train set
    ts_hat_val_list = trainBasicModels().train(basic_models, ts_train_for_val, frcy).predict(val_periods)
    ts_hat_list = trainBasicModels().train(basic_models, ts_list_train, frcy).predict(test_periods)
    
    # Subsets of data
    n_ts = len(ts_list_train)
    #sample_sizes.reverse()
    k_samples = [round(n_ts*ss) for ss in sample_sizes]
    #print(k_samples)
    seed = 202002020
    
    
    
    indexes_sample = {}
    
    for idx, (ss, k_sample) in enumerate(zip(sample_sizes, k_samples)):
        #if idx == 0:
        random.seed(seed+idx)
        indexes_sample[str(ss)] = random.sample(range(n_ts), k=k_sample)
        #    bef_sample = indexes_sample[str(ss)]
            #print(bef_sample)
        #else:
        #    indexes_sample[str(ss)] = random.sample(bef_sample, k=k_sample)
        #    bef_sample = indexes_sample[str(ss)]
            #print(bef_sample)
    
    #sample_sizes.reverse()
    samples = {}
    samples['complete'] = {
        'ts': ts_list_train, 'ts_test': ts_list_test, 'ts_hat_test': ts_hat_list,
        'ts_val': ts_val_list, 'ts_hat_val': ts_hat_val_list
    }
    #print(len(ts_hat_list))
    for sample in sample_sizes:
        idxs = indexes_sample[str(sample)]
        #print(idxs)
        samples[f'sample_{sample}'] = {
            'ts': [ts_list_train[idx] for idx in idxs], 
            'ts_test': [ts_list_test[idx] for idx in idxs], 
            'ts_hat_test': [ts_hat_list[idx] for idx in idxs],
            'ts_val': [ts_val_list[idx] for idx in idxs], 
            'ts_hat_val': [ts_hat_val_list[idx] for idx in idxs]
        }
        
    return samples
    

In [3]:
sample_sizes = [.1, .1, .1, .1, .1]
frcy = 12

In [4]:
m4 = get_data(data='M4', frcy=frcy, sample_sizes=sample_sizes)

100%|██████████| 48000/48000 [00:01<00:00, 36508.51it/s]


In [5]:
m3 = get_data(data='M3', frcy=frcy, sample_sizes=sample_sizes)

100%|██████████| 1428/1428 [00:00<00:00, 33678.77it/s]


In [6]:
owas = []
complete_m3 = m3['complete']

for idx, sample in enumerate(sample_sizes):
    print('\n \n')
    print(f'========Sample {idx}: {sample}===========')
    name_sample = 'sample_' + str(sample)
    
    print('===========Training on M4===========')
    m4_sample = m4[name_sample]
    fforma_m4 = FForma(m4_sample['ts'], m4_sample['ts_val'], m4_sample['ts_hat_val'], m4_sample['ts_hat_test'], frcy).train()
    
    print('===========Training on M3===========')
    m3_sample = m3[name_sample]
    fforma_m3 = FForma(m3_sample['ts'], m3_sample['ts_val'], m3_sample['ts_hat_val'], m3_sample['ts_hat_test'], frcy).train()
    
    print('===========Predicting with M4===========')
    preds_m3_m4 = fforma_m4.predict(complete_m3['ts'], complete_m3['ts_hat_test'], frcy=frcy)
    preds_m3_m4 = [[pred] for pred in preds_m3_m4]
    
    owa_m3_m4 = fforma_m4.contribution_to_owa(complete_m3['ts'], complete_m3['ts_test'], preds_m3_m4, frcy).mean()
    
    print('===========Predicting with M3===========')
    preds_m3_m3 = fforma_m3.predict(complete_m3['ts'], complete_m3['ts_hat_test'], frcy=frcy)
    preds_m3_m3 = [[pred] for pred in preds_m3_m3]
    owa_m3_m3 = fforma_m3.contribution_to_owa(complete_m3['ts'], complete_m3['ts_test'], preds_m3_m3, frcy).mean()
    
    df = pd.DataFrame({'idx': idx, 'sample': sample, 'owa_m3_m4': owa_m3_m4, 'owa_m3_m3': owa_m3_m3}, index=[0]) 
    print('Errors:')
    print(df)
    
    owas.append(df)

  1%|▏         | 67/4800 [00:00<00:07, 663.05it/s]


 

========Sample 0: 0.1===========
===========Training on M4===========
Setting model
Computing contribution to owa
Training NAIVE2


100%|██████████| 4800/4800 [00:06<00:00, 734.31it/s]
3753it [00:00, 18704.61it/s]

Calculating errors


4800it [00:00, 18632.35it/s]
4800it [00:00, 7470.01it/s]
4800it [00:00, 51628.56it/s]
2015it [00:00, 20142.48it/s]

...

  4%|▍         | 60/1428 [00:00<00:02, 596.76it/s]

Training NAIVE2


100%|██████████| 1428/1428 [00:02<00:00, 567.75it/s]
0it [00:00, ?it/s]

Calculating errors


AssertionError: ts must have the same size of ts_hat

In [14]:
print('\nComplete M3')
#fforma_m3_comp = FForma(complete_m3['ts'], complete_m3['ts_val'], complete_m3['ts_hat_val'], complete_m3['ts_hat_test'], frcy).train()
preds_m3_comp = fforma_m3_comp.predict(complete_m3['ts'], complete_m3['ts_hat_test'], frcy=frcy)
preds_m3_comp = [[pred] for pred in preds_m3_comp]
owa_m3_m3_comp = fforma_m3_comp.contribution_to_owa(complete_m3['ts'], complete_m3['ts_test'], preds_m3_comp, frcy).mean()
df = pd.DataFrame({'idx': len(sample_sizes)+1, 'sample': 1, 'owa_m3_m4': 0, 'owa_m3_m3': owa_m3_m3_comp}, index=[0]) 
print('Errors:')
print(df)

owas.append(df)


Complete M3





  0%|          | 0/1428 [00:00<?, ?it/s]


  4%|▍         | 63/1428 [00:00<00:02, 623.09it/s]

Training NAIVE2





  8%|▊         | 116/1428 [00:00<00:02, 590.79it/s]


 12%|█▏        | 173/1428 [00:00<00:02, 582.29it/s]


 15%|█▌        | 221/1428 [00:00<00:02, 545.26it/s]


 20%|██        | 292/1428 [00:00<00:01, 585.34it/s]


 25%|██▌       | 357/1428 [00:00<00:01, 601.65it/s]


 30%|██▉       | 427/1428 [00:00<00:01, 626.24it/s]


 38%|███▊      | 543/1428 [00:00<00:01, 725.73it/s]


 46%|████▌     | 651/1428 [00:00<00:00, 802.35it/s]


 52%|█████▏    | 736/1428 [00:01<00:00, 699.54it/s]


 57%|█████▋    | 812/1428 [00:01<00:00, 646.00it/s]


 62%|██████▏   | 882/1428 [00:01<00:00, 610.05it/s]


 66%|██████▋   | 947/1428 [00:01<00:00, 575.69it/s]


 71%|███████   | 1008/1428 [00:01<00:00, 569.07it/s]


 75%|███████▍  | 1068/1428 [00:01<00:00, 558.09it/s]


 79%|███████▉  | 1126/1428 [00:01<00:00, 544.75it/s]


 83%|████████▎ | 1182/1428 [00:01<00:00, 543.59it/s]


 87%|████████▋ | 1238/1428 [00:02<00:00, 548.36it/s]


 91%|█████████ | 1299/1428 [00:02<00:00, 564.13it/s]


 95%|█████████▍| 13

Calculating errors
Naive 2 errors
Errors:
   idx  sample  owa_m3_m4  owa_m3_m3
0    6       1          0    0.89748


In [15]:
owas_df = pd.concat(owas)
#owas_df['sample'] = 100*owas_df['sample']

In [27]:
full_m3 = owas_df[(owas_df['idx']==6)]['owa_m3_m3'][0]

In [29]:
full_m3

0.897479723520485

In [30]:
owas_df_plot = owas_df[owas_df['idx']<6]

In [36]:
owas_df_plot['idx'] = owas_df_plot['idx'] + 1

/home/fede/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
ax = owas_df_plot.set_index('idx').drop(columns='sample').plot(kind='bar')
plt.axhline(full_m3)
plt.title('OWA for M3 predictions (monthly) with FFORMA \n Using M4 and M3 training sets')
plt.xlabel('Random samples')
plt.legend(['OWA using full M3 to predict M3', 'OWA using 10% of M4 to predict full M3', 'OWA using 10% of M3 to predict fulll M3'], loc='lower center')